In [1]:
from IPython.display import Image, display, HTML
import requests
import json
import os
import base64

pp = lambda x: print(json.dumps(x, indent=2))

SKEMA_ADDRESS = os.environ.get("SKEMA_ADDRESS", "https://api.askem.lum.ai/isa/align-eqns")

## Aligning two equations in presentation MathML

In [2]:
# Define two mathematical equations in MathML format
halfar_dome_eqn1 = """<math>
      <mfrac>
        <mrow>
          <mi>&#x2202;</mi>
          <mi>H</mi>
        </mrow>
        <mrow>
          <mi>&#x2202;</mi>
          <mi>t</mi>
        </mrow>
      </mfrac>
      <mo>=</mo>
      <mi>&#x2207;</mi>
      <mo>&#x22C5;</mo>
      <mo>(</mo>
      <mi>&#x0393;</mi>
      <msup>
        <mi>H</mi>
        <mrow>
          <mi>n</mi>
          <mo>+</mo>
          <mn>2</mn>
        </mrow>
      </msup>
      <mo>|</mo>
      <mi>&#x2207;</mi>
      <mi>H</mi>
      <msup>
        <mo>|</mo>
        <mrow>
          <mi>n</mi>
          <mo>&#x2212;</mo>
          <mn>1</mn>
        </mrow>
      </msup>
      <mi>&#x2207;</mi>
      <mi>H</mi>
      <mo>)</mo>
    </math>
    """

In [3]:
# Equation 2: Remove <mi>&#x0393;</mi> (Γ) from the last equation
halfar_dome_eqn2 = """<math>
      <mfrac>
        <mrow>
          <mi>&#x2202;</mi>
          <mi>H</mi>
        </mrow>
        <mrow>
          <mi>&#x2202;</mi>
          <mi>t</mi>
        </mrow>
      </mfrac>
      <mo>=</mo>
      <mi>&#x2207;</mi>
      <mo>&#x22C5;</mo>
      <mo>(</mo>
      <msup>
        <mi>H</mi>
        <mrow>
          <mi>n</mi>
          <mo>+</mo>
          <mn>2</mn>
        </mrow>
      </msup>
      <mo>|</mo>
      <mi>&#x2207;</mi>
      <mi>H</mi>
      <msup>
        <mo>|</mo>
        <mrow>
          <mi>n</mi>
          <mo>&#x2212;</mo>
          <mn>1</mn>
        </mrow>
      </msup>
      <mi>&#x2207;</mi>
      <mi>H</mi>
      <mo>)</mo>
    </math>
    """

In [4]:
# Optional: Mention file directories for each equation
mention_json1_content = ""  # Mention file directory for Eqn 1 (optional)
mention_json2_content = ""  # Mention file directory for Eqn 2 (optional)

In [5]:
# Prepare data for the POST request
data = {
    "mml1": halfar_dome_eqn1,
    "mml2": halfar_dome_eqn2,
    "mention_json1": mention_json1_content,
    "mention_json2": mention_json2_content,
}

In [6]:
# Make a POST request to the Skema service
r = requests.post(SKEMA_ADDRESS, params=data)

In [7]:
# Print the matching ratio and union graph from the response
print(json.loads(r.text)["matching_ratio"])
print(json.loads(r.text)["union_graph"])

0.94
digraph G {
0 [color=blue, label="Div(Γ*(H^(n+2))*(Abs(Grad(H))^(n-1))*Grad(H)) <<|>> Div((H^(n+2))*(Abs(Grad(H))^(n-1))*Grad(H))"];
1 [color=blue, label="PD(1, t)(H)"];
2 [color=blue, label="Γ*(H^(n+2))*(Abs(Grad(H))^(n-1))*Grad(H) <<|>> (H^(n+2))*(Abs(Grad(H))^(n-1))*Grad(H)"];
3 [color=red, label="Γ"];
4 [color=blue, label="H^(n+2)"];
5 [color=blue, label="H"];
6 [color=blue, label="n+2"];
7 [color=blue, label="n"];
8 [color=blue, label="2"];
9 [color=blue, label="Abs(Grad(H))^(n-1)"];
10 [color=blue, label="Abs(Grad(H))"];
11 [color=blue, label="Grad(H)"];
12 [color=blue, label="n-1"];
13 [color=blue, label="1"];
1 -> 0  [color=blue, label="="];
2 -> 0  [color=blue, label="Div"];
3 -> 2  [color=red, label="*"];
4 -> 2  [color=blue, label="*"];
5 -> 4  [color=blue, label="^"];
6 -> 4  [color=blue, label="^"];
7 -> 6  [color=blue, label="+"];
8 -> 6  [color=blue, label="+"];
9 -> 2  [color=blue, label="*"];
10 -> 9  [color=blue, label="^"];
11 -> 10  [color=blue, label="Abs"];
5

## Aligning code and equation
We use the code for the SIR model and the equation I in SIR as an example.

In [8]:
SKEMA_LOCAL_ADDRESS = "https://api.askem.lum.ai"

In [9]:
single_snippet_payload = {
    "system": {
        "files": ["code.py"],
        "blobs": [
            'def sir(\n    s: float, i: float, r: float, beta: float, gamma: float, n: float\n) -> Tuple[float, float, float]:\n    """The SIR model, one time step."""\n    s_n = (-beta * s * i) + s\n    i_n = (beta * s * i - gamma * i) + i\n    r_n = gamma * i + r\n    scale = n / (s_n + i_n + r_n)\n    return s_n * scale, i_n * scale, r_n * scale'
        ],
    },
    "mml": """<math>
  <mfrac>
    <mrow>
      <mi>d</mi>
      <mi>I</mi>
    </mrow>
    <mrow>
      <mi>d</mi>
      <mi>t</mi>
    </mrow>
  </mfrac>
  <mo>=</mo>
  <mfrac>
    <mrow>
        <mi>&#x03B2;</mi>
        <mi>I</mi>
        <mi>S</mi>
    </mrow>
    <mi>N</mi>
  </mfrac>
  <mo>&#x2212;</mo>
  <mi>&#x03B3;</mi>
  <mi>I</mi>
</math>""",
}

The output consists of a JSON response containing aligned equation and code information. The structure of the response includes:
* Matching Ratio: Indicates the similarity between the equations.
* Number of Different Edges: Represents the dissimilarity between the equations.
* Node Labels Lists: Names of variables and terms in the equations.
* Aligned Indices Lists: Indices indicating alignment between equation components.
* Union Graph Visualization: Graphical representation of the alignment result.
* Perfectly Matched Indices: Indices of strictly matched nodes in Graph 1 of equation 1.

In [10]:
URL = f"{SKEMA_LOCAL_ADDRESS}/workflows/isa/code-eqn-align"
response = requests.post(URL, json=single_snippet_payload)
print(response.json())

{'3': [0.36, 7.0, ['β*I*S/N-γ*I', 'D(1, t)(I)', 'β*I*S/N', 'β', 'I', 'S', 'N', 'γ*I', 'γ'], ['i*-(beta)*s+s', 'D(1, t)(s)', 'i*-(beta)*s', 'i', '-(beta)', 'beta', 's'], [0.0, 1.0, 2.0, 5.0, 3.0, 6.0, 4.0, -1.0, -1.0], [0.0, 1.0, 2.0, 4.0, 6.0, 3.0, 5.0, -1.0, -1.0], 'digraph G {\n0 [color=blue, label="β*I*S/N-γ*I <<|>> i*-(beta)*s+s"];\n1 [color=blue, label="D(1, t)(I) <<|>> D(1, t)(s)"];\n2 [color=blue, label="β*I*S/N <<|>> i*-(beta)*s"];\n3 [color=blue, label="β <<|>> beta"];\n4 [color=blue, label="I"];\n5 [color=blue, label="S"];\n6 [color=blue, label="N <<|>> -(beta)"];\n7 [color=red, label="γ*I"];\n8 [color=red, label="γ"];\n1 -> 0  [color=blue, label="="];\n2 -> 0  [color=blue, label="+"];\n3 -> 2  [color=red, label="*"];\n4 -> 2  [color=blue, label="*"];\n5 -> 2  [color=blue, label="*"];\n6 -> 2  [color=red, label="/"];\n7 -> 0  [color=red, label="-"];\n8 -> 7  [color=red, label="*"];\n4 -> 7  [color=red, label="*"];\n6 -> 2  [color=green, label="*"];\n3 -> 6  [color=green, labe